# 1. Introduction

This notebook is used for mapping land cover, at a national scale for Greece, using Sentinel-2 composites (with 4 bands) and a weakly supervised training set from CORINE Land Cover data. Preprocessing of satellite data and reference samples is performed in Google Earth Engine and locally. The model used is the Fine Grained UNET model by  Stoian et al., 2019 (https://www.mdpi.com/2072-4292/11/17/1986), and a weighted categorical crossentropy loss.

*creator: Natalia Verde, AUTH, 2023*

## Package versions


* Tensorflow version: 2.11.0
* Keras version: 2.11.0
* h5py version: 3.1.0
* Folium version: 0.12.1.post1
* Matplotlib version: 3.2.2




## View resources

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# 2. Libraries, imports and authentications

Authenticate and import as necessary.

In [ ]:
print('Authenticate and initialize Colab ...')

# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

print("----------")
print("done!")

In [ ]:
print('Authenticate and initialize Earth Engine ...')

# Import, authenticate and initialize the Earth Engine library.

try:
  import ee
  ee.Initialize()
except:
  ee.Authenticate() # In colab only run once per week - not needed for runing in Jupyter
finally:
  import ee
  ee.Initialize()

print("----------")
print("done!")

Connect to Google Drive
(not used when running code locally in jupyter)

If you have exported the samples in Google Drive, access them from there.

see: https://stackoverflow.com/questions/57419346/how-can-i-access-my-google-drive-files-from-google-colab

In [ ]:
print("Connecting to drive ...")

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

print("----------")
print("done!")

In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
from datetime import datetime
from google.cloud import storage
import pickle

import tensorflow as tf
print("Temsorflow version:")
print(tf.__version__)

import keras
print("Keras version:")
print(keras.__version__)

import h5py
print("h5py version:")
print(h5py.__version__)

import folium
print("Folium version:")
print(folium.__version__)

import matplotlib
print("Matplotlib version:")
print(matplotlib.__version__)
import matplotlib.pyplot as plt

Temsorflow version:
2.12.0
Keras version:
2.12.0
h5py version:
3.8.0
Folium version:
0.14.0
Matplotlib version:
3.7.1


# 3. Set global variables

In [ ]:
print("Setting global variables ...")

# Specify names locations for outputs.

BASE_FOLDER = "/content/drive/My Drive/"  ########## FOR RUNNING WITH GOOGLE DRIVE ##########
BUCKET = "n-verde_bucket_1542"  ########## FOR RUNNING WITH GOOGLE CLOUD ##########
FOLDER = "1542_CLC_new"

# storing in Drive or Google Cloud?
# STORAGE = 'DRIVE'
STORAGE = 'GC'

# ------------------------------------------------------------------------------
# ---------------------- FEATURES --------------------------

# Specify inputs (Composite bands) to the model and the response variable.

otherData = [
              # 'DEM',
              'D0501NDVI', 'D0801NDVI', # 'D1001NDVI'
             ]

BANDS = [
          'D0501B2', 'D0501B3', 'D0501B4', 'D0501B8',
          'D0801B2', 'D0801B3', 'D0801B4', 'D0801B8',
          # 'D1001B2', 'D1001B3', 'D1001B4', 'D1001B8'
          ] + otherData

RESPONSE = ['gt']

CLASS_NUMBER = 6 # 1.Forest, 2.Grassland, 3.Cropland, 4.Wetland, 5.Settlement, 6.Other (+ 7.Background/NULL)

# Specify the number of tiles the imagery will be split in
# Predictions will run per tile
S = 10 # split to SxS tiles
# S = 100 # split to SxS tiles

# pixel size of image for export, training & prediction (in meters)
pixelSize = 20

# ------------------------------------------------------------------------------
# ---------------------- SAMPLES --------------------------

TRAINING_BASE = 'tra'
VAL_BASE = 'val'
TEST_BASE = 'test'

TRAIN_SIZE =  10000    # 6 samples per polygon * 5 augmentations = 7000 * 5 = 35,000
VAL_SIZE =    10000    # 6 samples per polygon * 5 augmentations = 7000 * 5 = 35,000
TEST_SIZE =   2604    # 6 samples per polygon # LIFE

# BATCH_SIZE is dependent on your GPU memory. (e.g. on my PC it can't be larger than 1, on Colab it can be 4)
BATCH_SIZE = 8
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
EPOCHS = 50 # (Stoian)

BUFFER_SIZE = 1500

# ---------------------- MODEL -------------------------

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in BANDS
]

BANDS_DICT = dict(zip(BANDS, COLUMNS))

RESPONSE_DICT = dict(zip(
      RESPONSE,
      [tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in RESPONSE]
))
# UNIQUES = np.array([0.   , 0.166, 0.333, 0.499, 0.666, 0.833, 1.   ], dtype="float32") # CLASS_NUMBER in one-hot (7)
UNIQUES = np.array([0.166, 0.333, 0.499, 0.666, 0.833, 1.   ], dtype="float32") # CLASS_NUMBER in one-hot (6)

lr = 0.0001 # initial learning rate (Scepanovic, Schmitt, Stoian 0.001)

# ---------- WEIGHTS FOR WEIGHTED CATEGORICAL CROSSENTROPY LOSS
# weights: numpy array of shape (C,) where C is the number of classes
# Usage: weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
# weights are the 100/percentage of class area in the samples...
weights = [
						1.39, # Forest
						1.26, # Grassland
						1.43, # Cropland
						80.55, # Wetland
						12.99, # Settlement
						2.38  # Other
					]

print("----------")
print("done!")

In [ ]:
# # #############  FOR TESTING ONLY #########################

# TRAIN_SIZE =  50
# VAL_SIZE =    25
# TEST_SIZE =   25
# BUFFER_SIZE = 100

# EPOCHS = 10

In [ ]:
# ------------------------------------------------------------------------------
# ---------------------- GEE (images, ground truth) --------------------------

# composite images to use for training and prediction
spring = ee.Image('users/n-verde/PhD_1542/MedianMonths5-6_GR_2018')
summer = ee.Image('users/n-verde/PhD_1542/MedianMonths7-9_GR_2018')
autumn = ee.Image('users/n-verde/PhD_1542/MedianMonths10-11_GR_2018')

# mountains
kapos = ee.Image('users/n-verde/PhD_1542/Kapos_K1_GR_binary')

# DEM
dem = ee.Image('users/n-verde/shared/LIFE-IP_4_NATURA/eudem_v11_GR_EPSG4326')

# ground truth
gt_tra = ee.Image('users/n-verde/PhD_1542/NEW_CLC2018_GR_4326_IPCC_one-hot_TRAIN_14perc')
gt_val = ee.Image('users/n-verde/PhD_1542/NEW_CLC2018_GR_4326_IPCC_one-hot_VAL_14perc')
gt_test = ee.Image('users/n-verde/PhD_1542/NEW_LIFE_4326_IPCC_one-hot_TEST_30perc') # LIFE

# ground truth BOXES VECTOR
trainingPolys = ee.FeatureCollection('users/n-verde/PhD_1542/NEW_Kapos_grid_5120m_4326_TRAIN_14perc')
valPolys = ee.FeatureCollection('users/n-verde/PhD_1542/NEW_Kapos_grid_5120m_4326_VAL_14perc')
testPolys = ee.FeatureCollection('users/n-verde/PhD_1542/NEW_LIFE_grid_TEST_30perc_random')

# 4. Imagery


Gather and setup the imagery to use for inputs (predictors).
Also use some pre-made geometries to sample the stack in strategic locations. Specifically, these are polygons in which to take the 256x256 samples. Display the sampling polygons on a map, red for training polygons, blue for validation, green for test polygons.

## Visualization

In [ ]:
# Use folium to visualize the imagery.
map = folium.Map(location=[37.981892554434936, 23.7269115882649])

# ------------------------------------------

# rename bands to display
renamedOpticalBands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7',
                'B8', 'B8A', 'B11', 'B12']

bandNames = summer.bandNames()
mos79 = summer.select(bandNames, renamedOpticalBands)

mapid = mos79.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 300, 'max': 2500})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='MedianMonths7-9',
  ).add_to(map)

# ------------------------------------------

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(valPolys, 2).paint(testPolys, 3)
polyImage = polyImage.updateMask(polyImage)

# display the sample boxes
mapid = polyImage.getMapId({'min': 1, 'max': 3, 'palette': ['red', 'blue', 'green']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training & test polygons',
  ).add_to(map)

# ------------------------------------------
# ground truth
gt = ee.ImageCollection([gt_tra, gt_val, gt_test]).mosaic()
gt = gt.rename('gt')

mapid = gt.getMapId({'bands': ['gt'], 'min': 0, 'max': 1})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='ground_truth',
  ).add_to(map)

map.add_child(folium.LayerControl())
map


## Spectral indices

In [ ]:
ndvi56 = spring.normalizedDifference(['D0501B8', 'D0501B4']).rename('D0501NDVI')
ndvi79 = summer.normalizedDifference(['D0801B8', 'D0801B4']).rename('D0801NDVI')
# ndvi1011 = autumn.normalizedDifference(['D1001B8', 'D1001B4']).rename('D1001NDVI')
# print(ndvi56.getInfo())

## Temporal composite

In [ ]:
mos56 = spring.select(['D0501B2', 'D0501B3', 'D0501B4', 'D0501B8']).addBands(ndvi56)
mos79 = summer.select(['D0801B2', 'D0801B3', 'D0801B4', 'D0801B8']).addBands(ndvi79)
# mos1011 = autumn.select(['D1001B2', 'D1001B3', 'D1001B4', 'D1001B8']).addBands(ndvi1011)

mos = mos56.addBands(mos79) #.addBands(mos1011)
# print(mos.getInfo())

In [ ]:
dem = dem.rename('DEM')
# print(dem.getInfo())

## Normalize
Rescale imagery to [0-1]

In [ ]:
############### HELPER FUNCTIONS ###############

def findMin(imgBand, bandName) :
     MIN = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': aoi,
    'scale': (pixelSize*2),
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

     return MIN


def findMax(imgBand, bandName) :
     MAX = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': aoi,
    'scale': (pixelSize*2),
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

     return MAX


def rescaleFixedMinMax(img, bandName, MIN, MAX):
     Min = ee.Number(MIN)
     Max = ee.Number(MAX)

     imgBand = img.select(bandName)
     imgBand = imgBand.float()
     imgBandNorm = imgBand.divide(Max) # normalize the data to 0 - 1

     return imgBandNorm


def rescaleBand(oneBandImg): # only works for an image with 1 band

   bandName = ee.String(oneBandImg.bandNames().get(0)) #print(bandName)

   imgBand = oneBandImg.select(bandName)

   MIN = findMin(imgBand, bandName) # print('old min', MIN.getInfo())
   MAX = findMax(imgBand, bandName) # print('old max', MAX.getInfo())

   test = ee.Number(ee.Algorithms.If(MIN.lt(ee.Number(0)), 1, 0)) #print(test)

   if (test.eq(ee.Number(1))) :
      imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
      MAX = MAX.add(MIN.abs()) # change new max
      MIN = ee.Number(0) # change min to 0

   min = MIN # print(ee.String("min of ").cat(bandName), min)
   max = MAX # print(ee.String("max of ").cat(bandName), max)

  # Apply the rescale def to all bands of the oneBandImg
   res = rescaleFixedMinMax(oneBandImg,bandName, min, max) #print(ee.Image(res))

   return ee.Image(res)


def bandsToCollection(image):
   collection = ee.ImageCollection.fromImages(image.bandNames().map(
       lambda bandName: image.select(ee.String(bandName))
      ))

   return collection


In [ ]:
############### NORMALIZE ###############

print("Rescaling optical imagery to 0-1 ...")

# define an aoi in which to compute min and max of image
aoi = ee.Image('users/n-verde/auth/PhenologyPaper/AMATH_buff100_raster_compressed').geometry()

# turn image to IC in order to .map
bandsInFormOfIC = bandsToCollection(mos)

# normalize to 0-1
rescaledBandsIC = ee.ImageCollection(bandsInFormOfIC.map(rescaleBand))

# merge the collection to a single image again
def col2img(image, previous):
  return ee.Image(previous).addBands(ee.Image(image))

rescaledBands = ee.Image(rescaledBandsIC.iterate(col2img, ee.Image()))

print("Rescaling DEM to 0-1 ...")

# turn image to IC in order to .map
bandsInFormOfIC = bandsToCollection(dem)

# normalize to 0-1
rescaledDEMIC = ee.ImageCollection(bandsInFormOfIC.map(rescaleBand))

# merge the collection to a single image again
def col2img(image, previous):
  return ee.Image(previous).addBands(ee.Image(image))

rescaledDEM = ee.Image(rescaledDEMIC.iterate(col2img, ee.Image()))

print("----------")
print("done!")

In [ ]:
image_res = rescaledBands.addBands(rescaledDEM)

# display rescaled image
mapid = image_res.getMapId({'bands': ['D0801NDVI'], 'min': 0, 'max': 1})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='rescaled',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Mask to  samples extent

Mask the rescaled composite image to the samples extent (boxes image).
Sample boxes were created based on:
* https://www.tandfonline.com/doi/full/10.1080/19475683.2020.1803402
* https://explore.openaire.eu/search/publication?pid=10.1109%2Fjstars.2021.3116094
* https://www.mdpi.com/2072-4292/10/8/1214


In [ ]:
boxes = ee.ImageCollection.fromImages([gt_tra, gt_val, gt_test]).mosaic().gte(0)
masked_image = image_res.mask(boxes)

# display the masked image
mapid = masked_image.getMapId({'bands': ['D0801B4', 'D0801B3', 'D0801B2'], 'min': 0, 'max': 0.25})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='mos_masked',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Split image to tiles
Define a function for displaying Earth Engine image tiles on a folium map

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# This function splits a geometry into equal-sized subrectangles.
# splits a geometry to parts^2 subregions
# Assumes that the geometry has only 4 vertices.
# returns a list of polygons (you can map over it)
def split(geom, nSplits):

  one = ee.Number(1)

  # Return (nSplit+1)^2 coordinate pairs, given 4 vertices.
  def toPts(rect, nSplits):
    k = ee.List.sequence(0, None, one.divide(nSplits), one.add(nSplits));

    def f1(x):

      def f2(y):
        xp = one.subtract(x)
        yp = one.subtract(y)
        coeffs = ee.Array([[xp.multiply(yp), yp.multiply(x), xp.multiply(y), ee.Number(x).multiply(y)]])
        return coeffs.matrixMultiply(rect).project([1])

      return k.map(f2)

    return k.map(f1).flatten()

  # Return nSplit^2 polygons, given the list of vertices built by toPts().
  def toRects(pts, nSplits):
    offsets = ee.List([0, 1, one.add(nSplits).add(one), one.add(nSplits)])
    k1 = ee.List.sequence(0, None, one.add(nSplits), nSplits)

    def f3(i):
      k2 = ee.List.sequence(i, None, 1, nSplits)

      def f4(j):

        def f5(offset):
          return ee.Array(pts.get(ee.Number(j).add(offset))).toList()

        return ee.Geometry.Polygon(offsets.map(f5))

      return k2.map(f4)

    return k1.map(f3).flatten()


  # Get the 4 vertices.  Assumes that the scene geometry has only 4 vertices.
  rect = ee.List(geom.coordinates().get(0))
  rect = ee.Array([rect.get(0), rect.get(1), rect.get(3), rect.get(2)])

  pts = toPts(rect, nSplits)
  rects = toRects(pts, nSplits)

  print('Parts that geometry was splitted to: ', ee.List(rects).size().getInfo())

  return ee.List(rects)

print("Splitting to tiles ...")

# get composite image bounds in a box
i = mos.geometry().bounds()

# split to SxS tiles
splitted = split(i,S)

print("----------")
print("done!")


In [ ]:
# composite mosaic image
image = mos79
mountains = image.mask(kapos)

def geom2feat(g):
    return ee.Feature(ee.Geometry(g))
features = splitted.map(geom2feat)
splits = ee.FeatureCollection(features)

tilesreduced = mountains.select('D0801B4').reduceRegions(**{
  'collection': splits,
  'reducer': ee.Reducer.mean(),
  'scale': 200,
})

tilesMountainsIntersec = ee.FeatureCollection(tilesreduced.filter(ee.Filter.notNull(['mean'])))

print('Tiles containing mountains: ', ee.FeatureCollection(tilesMountainsIntersec).size().getInfo())

In [ ]:
# show on Map

mapid = mountains.getMapId({'bands': ['D0801B4', 'D0801B3', 'D0801B2'], 'min': 300, 'max': 2500})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='composite',
  ).add_to(map)

# composite bounds
# map.add_ee_layer(i, {}, 'composite bounds')

no = 25
lis = tilesMountainsIntersec.toList(ee.FeatureCollection(tilesMountainsIntersec).size().getInfo())
spl = ee.Geometry(ee.List(lis.get(no)))
desc = 'tile ' + str(int(no)) + ' from the split'

# tile no
map.add_ee_layer(spl, {}, desc)

# # all tiles
# map.add_ee_layer(splits, {}, "splits")

# tiles with mountains
map.add_ee_layer(tilesMountainsIntersec, {}, "intersection")

map.add_child(folium.LayerControl())
map

# 5. Sampling

## Convert images to arrays

Convert the imagery and GT into an array image in which each pixel stores ...x... patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [ ]:
features = ee.Image(masked_image.select(BANDS)).float()
GT = ee.Image(gt.select(RESPONSE)).float()

print('features: ', features.bandNames().getInfo())
print('GT: ', GT.bandNames().getInfo())

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arraysFeatures = features.neighborhoodToArray(kernel)
arraysGT = GT.neighborhoodToArray(kernel)

print("----------")
print("done!")

Take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the CNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.
For whole Greece takes around **1.5h** in GEE, and around 10GB with the current sample and shard sizes.

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
valPolysList = valPolys.toList(valPolys.size())
testPolysList = testPolys.toList(testPolys.size())

# These numbers determined experimentally.
n = 2 # Number of shards in each polygon.
N =  6 # Total sample size in each polygon. (N 256x256 patches)

In [ ]:
print("Creating sample patches and exporting ...")

# -----------------------------------------------------------------------
#  ---------- 1. Export features (imagery - X) ----------

# # Export all the training data. -----------
# # (in many pieces), with one task per geometry.

# for g in range(trainingPolys.size().getInfo()):
# # for g in range(46,trainingPolys.size().getInfo()):
#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysFeatures.sample(
#             region = ee.Feature(trainingPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n, # Size of the shard.
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'X_' + TRAINING_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):
#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage(  ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()



# # Export all the validation data. ----------

# for g in range(valPolys.size().getInfo()):
# # for g in range(772,trainingPolys.size().getInfo()):
#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysFeatures.sample(
#             region = ee.Feature(valPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n,
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'X_' + VAL_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):

#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage( ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()


# # Export all the test data. ----------

# for g in range(testPolys.size().getInfo()):
# # for g in range(202,testPolys.size().getInfo()):
#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysFeatures.sample(
#             region = ee.Feature(testPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n,
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'X_' + TEST_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):
#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage( ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = BANDS
#       )
#       task.start()


# print("----------")
# print("done exporting features (X_tra, X_val, X_test)")

In [ ]:
print("Creating GT sample patches and exporting ...")

# #-----------------------------------------------------------------------
# # ---------- 2. Export GT (ground truth - y) ----------

# # Export all the training data. -----------

# for g in range(trainingPolys.size().getInfo()):
# # for g in range(844,trainingPolys.size().getInfo()):
#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysGT.sample(
#             region = ee.Feature(trainingPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n, # Size of the shard.
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'y_' + TRAINING_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):
#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage(  ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()

# # Export all the validation data. ----------

# for g in range(valPolys.size().getInfo()):
# # for g in range(533,valPolys.size().getInfo()):
# # because of error "Too many tasks already in the queue (3000). Please wait for some of them to complete."

#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysGT.sample(
#             region = ee.Feature(valPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n, # Size of the shard.
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'y_' + VAL_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):
#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage(  ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()

# # Export all the test data. ----------

# for g in range(testPolys.size().getInfo()):
# # for g in range(328,testPolys.size().getInfo()):
# # because of error "Too many tasks already in the queue (3000). Please wait for some of them to complete."

#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#         sample = arraysGT.sample(
#             region = ee.Feature(testPolysList.get(g)).geometry(),
#             scale = pixelSize,
#             numPixels = N / n, # Size of the shard.
#             seed = i,
#             tileScale = 16
#         )
#         geomSample = geomSample.merge(sample)

#     desc = 'y_' + TEST_BASE + '_g' + str(g)

#     if (STORAGE == 'DRIVE'):
#       task = ee.batch.Export.table.toDrive( ############ EXPORTS TO DRIVE ############
#           collection = geomSample,
#           description = desc,
#           folder = FOLDER,
#           fileNamePrefix = desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()
#     else:
#       task = ee.batch.Export.table.toCloudStorage(  ############ EXPORTS TO CLOUD STORAGE ############
#           collection = geomSample,
#           description = desc,
#           bucket = BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = RESPONSE
#       )
#       task.start()

# print("----------")
# print("done exporting ground truth (y_tra, y_val, y_test)")

# 6. Load sample Data

Download data from Google Drive and load the data exported from Earth Engine into a `tf.data.Dataset`.
Loading takes around **30m**

### Helper functions for reading training, validation and test data

In [ ]:
import tensorflow.python.ops.numpy_ops.np_config as np_config
np_config.enable_numpy_behavior()

def to_single(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

def to_multichannel(inputs):
    # inputsList = [inputs.get(key) for key in RESPONSE_DICT.keys()]
    # dim = tf.stack(inputsList, axis=0)
    dim = inputs.get(RESPONSE[0])
    label_channels = []    # or maybe a dict?
    for ch in UNIQUES:    # iterate through each desired label of pixels (found above through the unique values in the images)
        label_channels.append( (dim==ch).astype(tf.int32) )    # create a channel for each label and store in a list
    stacked_label = tf.transpose(tf.stack(label_channels), [1, 2, 0])    # stack them in a single
    return stacked_label

def get_labeled_dataset(prefix, driveTrue):

  if (STORAGE == 'DRIVE'): # if samples are stored in Drive
    pattern = BASE_FOLDER + FOLDER + '/' + '*'
  else: # if samples are stored in Google Cloud
    pattern =  'gs://' + BUCKET + '/' + FOLDER + '/' + '*'

  glob = tf.io.gfile.glob(pattern)

  x_glob = [x for x in glob if x.split("/")[-1].split("_")[0]=="X" and x.split("/")[-1].split("_")[1]==prefix]
  x_dataset = tf.data.TFRecordDataset(x_glob, compression_type='GZIP')
  x_dataset = x_dataset.map(
        lambda x: tf.io.parse_single_example(x, features=BANDS_DICT),
        num_parallel_calls=5
  )
  x_dataset = x_dataset.map(to_single, num_parallel_calls=5)

  y_glob = [x for x in glob if x.split("/")[-1].split("_")[0]=="y" and x.split("/")[-1].split("_")[1]==prefix]
  y_dataset = tf.data.TFRecordDataset(y_glob, compression_type='GZIP')
  y_dataset = y_dataset.map(
        lambda x: tf.io.parse_single_example(x, features=RESPONSE_DICT),
        num_parallel_calls=5
  )
  y_dataset = y_dataset.map(to_multichannel, num_parallel_calls=5)

  dataset = tf.data.Dataset.zip((x_dataset, y_dataset))

  if (prefix == "tra"):
    dataset = dataset.map(
                            lambda image, label: (tf.image.flip_left_right(image), label)
                      ).map(
                            lambda image, label: (tf.image.flip_up_down(image), label)
                      ).map(
                            lambda image, label: (tf.image.rot90(image), label) # rotate 90 degr
                      ).map(
                            lambda image, label: (tf.image.rot90(image, k=-2), label) # rotate 180 degr
                      ).map(
                            lambda image, label: (tf.image.rot90(image, k=3), label) # rotate 270 degr
                      ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
  elif (prefix == "val"):
    dataset = dataset.map(
                            lambda image, label: (tf.image.flip_left_right(image), label)
                      ).map(
                            lambda image, label: (tf.image.flip_up_down(image), label)
                      ).map(
                            lambda image, label: (tf.image.rot90(image), label) # rotate 90 degr
                      ).map(
                            lambda image, label: (tf.image.rot90(image, k=-2), label) # rotate 180 degr
                      ).map(
                            lambda image, label: (tf.image.rot90(image, k=3), label) # rotate 270 degr
                      ).batch(1).repeat()
  elif (prefix == "test"):
    dataset = dataset.batch(1).repeat()

  return dataset


### Training data

Use the helpers to read in the training dataset.  Print the first record to check.

In [ ]:
training = get_labeled_dataset("tra", False)

print("Inspecting training dataset's dimensions: ")
for x in training.take(1):
    print(x[0].numpy().shape)
    print(x[1].numpy().shape)

print(training)

### Validation and Test data

Now do the same thing to get an validation and test dataset.  Note that unlike the training dataset, the validation and test dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
validation = get_labeled_dataset("val", False)

print("Inspecting validation dataset's dimensions: ")
for x in validation.take(1):
    print(x[0].numpy().shape)
    print(x[1].numpy().shape)

print(validation)


In [ ]:
test = get_labeled_dataset("test", False)

print("Inspecting test dataset's dimensions: ")
for x in test.take(1):
    print(x[0].numpy().shape)
    print(x[1].numpy().shape)

print(test)

# 7. Existing model definition
Model acquired from Stoian et al., 2019, https://www.mdpi.com/2072-4292/11/17/1986
here: https://github.com/vpoughon/RT_DL_OSO_public

### Model
"get_unet_mlp_ts" model from here: https://github.com/vpoughon/RT_DL_OSO_public/blob/master/model_definitions.py

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Concatenate, core, Dropout, concatenate, Cropping2D, Convolution2D, ConvLSTM2D, Conv3D

def get_unet_mlp_ts(n_ch, n_timesteps, n_output, patch_height, patch_width,hidden_layers):
# create u-net model

    inputs = Input((patch_height, patch_width, n_ch))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up1 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    #
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    unet_model = Model(inputs=inputs, outputs=conv5)
    #

    ###

    l_prev = Conv2D(hidden_layers[0], (1, 1), activation='relu', padding='same')(inputs)
    for n_hidden_neurons in hidden_layers[1:]:
        fc_new = Conv2D(n_hidden_neurons, (1, 1), activation='relu', padding='same')(l_prev)
        # added by VP to decrease overfitting:
        fc_new = Dropout(0.2)(fc_new)
        l_prev = fc_new
    mlp_model = Model(inputs=inputs, outputs=l_prev)

    # share this model among temporal samples
    inputs_list = []
    out_list = []
    out_mlp_list = []
    for i in range(n_timesteps):
        input_patch = Input((patch_height, patch_width, n_ch))
        inputs_list.append(input_patch)
        out_unet_layer = unet_model(input_patch)
        out_mlp_layer = mlp_model(input_patch)
        out_list.append(out_unet_layer)
        out_mlp_list.append(out_mlp_layer)

    out_unet = concatenate(out_list,axis=-1)
    out_mlp = concatenate(out_mlp_list,axis=-1)

    out_concat = concatenate([out_mlp, out_unet],axis=-1)
    out_concat = core.Reshape(((patch_height,patch_width, hidden_layers[-1]+32)*n_timesteps))(out_concat)

    # create common layers for softmax
    out2 = Conv2D(n_output, (1, 1), activation='relu',padding='same')(out_concat)
    # !!!! KEEP THIS RESHAPE FOR LOSS IN TRAINING !!!!
    # !!!! COMMENT OUT AND COMPILE AGAIN FOR PREDICTION !!!!
    out2 = core.Reshape((patch_height*patch_width, n_output))(out2)
    out2 = core.Activation('softmax')(out2)

    # get unified model
    model_out = Model(inputs_list,out2)

    return model_out


### Loss

In [ ]:
from keras import backend as K
from keras.layers import Reshape, core

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

In [ ]:
# ******************** LOSS ***********************

if weights is not None:
		LOSS = weighted_categorical_crossentropy(weights)
else:
		LOSS='categorical_crossentropy'


# ---------- DICE LOSS ----------
# dice_loss = sm.losses.DiceLoss()
# LOSS = dice_loss

### Accuracy metrics

In [ ]:
# # -----------------------------------------

# iou = sm.metrics.IOUScore(threshold=0.5)
# fscore = sm.metrics.FScore(threshold=0.5)

# -----------------------------------------

def recall_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# -----------------------------------------
# Stoian et al. ----->
# 1
def acc_labeled_only(y_true, y_pred):
    '''
    accuracy evaluated only on labeled pixels
    '''
    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    annot_mask = K.cast(K.not_equal(K.max(y_true, axis=-1), 0), 'int64')
    num_annot = K.sum(annot_mask)
    num_not_annot = K.sum(1 - annot_mask)
    y_true_label = K.argmax(y_true, axis=-1)
    y_pred_label = K.argmax(y_pred, axis=-1)

    # all not annotated GT will be 0
    y_pred_all = (y_pred_label + K.ones_like(y_pred_label)) * annot_mask
    # all not annotated PRED will be 0, first valid class=1
    y_true_all = (y_true_label + K.ones_like(y_true_label)) * annot_mask

    # 0 = classes do not match, else 1. will be 1 for non annotated pixels
    mask_class_ok = K.cast(K.equal(y_pred_all, y_true_all), 'int64')

    # count all matching, subtract non annotated, divide by number of annotated
    return K.cast(K.sum(mask_class_ok) - num_not_annot, 'float32') / K.cast(num_annot, 'float32')

# -----------------------------------------

METRICS = [acc_labeled_only]


## Build model (compile)

In [ ]:
print("compiling model...")

OPTIMIZER = tf.keras.optimizers.Adam(lr) # (Stoian, Schmitt, Scepanovic, Hua)
# OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

# ********************* function to initialize model ********************

def get_model():
	# get_unet_mlp_ts(n_ch, n_timesteps, n_output, patch_height, patch_width, hidden_layers)
	m = get_unet_mlp_ts(len(BANDS),1,CLASS_NUMBER,KERNEL_SIZE,KERNEL_SIZE,[200,100,50])
	m.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
	return m

m_stoian = get_model()
m_stoian.summary()

model = m_stoian

print("...done!")


# 8. Train

## Set callbacks

In [ ]:
# path for best model
filename = 'best_model-STOIAN-epoch{epoch:02d}-loss{loss:.4f}-vloss{val_loss:.4f}-acc{acc_labeled_only:.4f}-vacc{val_acc_labeled_only:.4f}'

if (STORAGE == 'DRIVE'):
  bpath = BASE_FOLDER + FOLDER + '/' + filename
else:
  bpath =  'gs://' + BUCKET + '/' + FOLDER + '/' + filename

# ---------------------- CALLBACKS --------------------------

# best model is saved in every checkpoint
save_model = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    filepath=bpath,
    save_weights_only=True,
    save_best_only=True)

# early stopping
stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience=5, # wait for 5 epochs
)

# # LR reducer
# from keras.callbacks import ReduceLROnPlateau
# lr_reducer = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.9,
#     cooldown=0,
#     patience=5,
#     min_lr=0.001)


## Train

In [ ]:
# ---------------------- TRAIN --------------------------

print("training model...")

start = datetime.now()

history=model.fit(
          x=training,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=validation,
          validation_steps=VAL_SIZE,
          # callbacks=[lr_reducer, save_model, stopping],
          callbacks=[save_model, stopping],
          steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE)
          )

# save history
hfilename = 'trainHistoryDict'

if (STORAGE == 'DRIVE'):
  hpath = BASE_FOLDER + FOLDER + '/' + hfilename
  with open(hpath, 'wb') as file_pi:
    pickle.dump(history, file_pi)
else:
  storage_client = storage.Client()
  bucket = storage_client.bucket(BUCKET) # bucket_name = "your-bucket-name"
  blob = bucket.blob('/' + FOLDER + '/' + hfilename) # blob_name = "storage-object-name"
  with blob.open("wb") as f: # Mode can be specified as wb/rb for bytes mode.
      pickle.dump(history, f)


duration = datetime.now()-start
print("...done!")
print("Training completed in time: ", duration)

## Load a model from latest checkpoint and continue training

In [ ]:
print("loading weights from trained model...")

# m_updated = get_model()

# get the latest (best) model saved

if (STORAGE == 'DRIVE'):
  path = BASE_FOLDER + FOLDER
else:
  path =  'gs://' + BUCKET + '/' + FOLDER

latest = tf.train.latest_checkpoint(path)
model.load_weights(latest)

print("...done!")

loading weights from trained model...
...done!


In [ ]:
EPOCHS_LEFT = 45

# resume the training where we left off
start = datetime.now()

history=model.fit(
          x=training,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS_LEFT,
          verbose=1,
          validation_data=validation,
          validation_steps=VAL_SIZE,
          # callbacks=[lr_reducer, save_model, stopping],
          callbacks=[save_model, stopping],
          steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE)
          )

# save history
hfilename = 'trainHistoryDict'

if (STORAGE == 'DRIVE'):
  hpath = BASE_FOLDER + FOLDER + '/' + hfilename
  with open(hpath, 'wb') as file_pi:
    pickle.dump(history, file_pi)
else:
  storage_client = storage.Client()
  bucket = storage_client.bucket(BUCKET) # bucket_name = "your-bucket-name"
  blob = bucket.blob('/' + FOLDER + '/' + hfilename) # blob_name = "storage-object-name"
  with blob.open("wb") as f: # Mode can be specified as wb/rb for bytes mode.
      pickle.dump(history, f)


duration = datetime.now()-start
print("...done!")
print("Training completed in time: ", duration)

# 9. Evaluate model

## Load a model from latest checkpoint

In [ ]:
print("loading weights from trained model...")

# m_updated = get_model()

# get the latest (best) model saved

if (STORAGE == 'DRIVE'):
  path = BASE_FOLDER + FOLDER
else:
  path =  'gs://' + BUCKET + '/' + FOLDER

latest = tf.train.latest_checkpoint(path)
model.load_weights(latest)

print("...done!")

## Evaluate

In [ ]:
from classification_models import models
import keras.backend as K

EVAL_METRICS = [f1_m, recall_m, precision_m]

# compile keras model with defined optimozer, loss and metrics
model.compile(keras.optimizers.Adam(lr), LOSS, EVAL_METRICS)

# evaluate the model

print("evaluating model...")

loss, f1_score, recall, precision = model.evaluate(x=test, steps=int(TEST_SIZE / BATCH_SIZE))

print('...done!')

# 10. Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.



## Model modified for prediction

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Concatenate, core, Dropout, concatenate, Cropping2D, Convolution2D, ConvLSTM2D, Conv3D

def get_unet_mlp_ts(n_ch, n_timesteps, n_output, patch_height, patch_width,hidden_layers):
# create u-net model

    inputs = Input((patch_height, patch_width, n_ch))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up1 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    #
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    unet_model = Model(inputs=inputs, outputs=conv5)
    #

    ###

    l_prev = Conv2D(hidden_layers[0], (1, 1), activation='relu', padding='same')(inputs)
    for n_hidden_neurons in hidden_layers[1:]:
        fc_new = Conv2D(n_hidden_neurons, (1, 1), activation='relu', padding='same')(l_prev)
        # added by VP to decrease overfitting:
        fc_new = Dropout(0.2)(fc_new)
        l_prev = fc_new
    mlp_model = Model(inputs=inputs, outputs=l_prev)

    # share this model among temporal samples
    inputs_list = []
    out_list = []
    out_mlp_list = []
    for i in range(n_timesteps):
        input_patch = Input((patch_height, patch_width, n_ch))
        inputs_list.append(input_patch)
        out_unet_layer = unet_model(input_patch)
        out_mlp_layer = mlp_model(input_patch)
        out_list.append(out_unet_layer)
        out_mlp_list.append(out_mlp_layer)

    out_unet = concatenate(out_list,axis=-1)
    out_mlp = concatenate(out_mlp_list,axis=-1)

    out_concat = concatenate([out_mlp, out_unet],axis=-1)
    out_concat = core.Reshape(((patch_height,patch_width, hidden_layers[-1]+32)*n_timesteps))(out_concat)

    # create common layers for softmax
    out2 = Conv2D(n_output, (1, 1), activation='relu',padding='same')(out_concat)
    # !!!! KEEP THIS RESHAPE FOR LOSS IN TRAINING !!!!
    # !!!! COMMENT OUT AND COMPILE AGAIN FOR PREDICTION !!!!
    # out2 = core.Reshape((patch_height*patch_width, n_output))(out2)
    # out2 = core.Activation('softmax')(out2)

    # get unified model
    model_out = Model(inputs_list,out2)

    return model_out


### Loss

In [ ]:
from keras import backend as K
from keras.layers import Reshape, core

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

In [ ]:
# ******************** LOSS ***********************

if weights is not None:
		LOSS = weighted_categorical_crossentropy(weights)
else:
		LOSS='categorical_crossentropy'


# ---------- DICE LOSS ----------
# dice_loss = sm.losses.DiceLoss()
# LOSS = dice_loss

### Accuracy metrics

In [ ]:
# # -----------------------------------------

# iou = sm.metrics.IOUScore(threshold=0.5)
# fscore = sm.metrics.FScore(threshold=0.5)

# -----------------------------------------

def recall_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):

    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# -----------------------------------------
# Stoian et al. ----->
# 1
def acc_labeled_only(y_true, y_pred):
    '''
    accuracy evaluated only on labeled pixels
    '''
    y_true = core.Reshape((KERNEL_SIZE*KERNEL_SIZE, CLASS_NUMBER))(y_true)

    annot_mask = K.cast(K.not_equal(K.max(y_true, axis=-1), 0), 'int64')
    num_annot = K.sum(annot_mask)
    num_not_annot = K.sum(1 - annot_mask)
    y_true_label = K.argmax(y_true, axis=-1)
    y_pred_label = K.argmax(y_pred, axis=-1)

    # all not annotated GT will be 0
    y_pred_all = (y_pred_label + K.ones_like(y_pred_label)) * annot_mask
    # all not annotated PRED will be 0, first valid class=1
    y_true_all = (y_true_label + K.ones_like(y_true_label)) * annot_mask

    # 0 = classes do not match, else 1. will be 1 for non annotated pixels
    mask_class_ok = K.cast(K.equal(y_pred_all, y_true_all), 'int64')

    # count all matching, subtract non annotated, divide by number of annotated
    return K.cast(K.sum(mask_class_ok) - num_not_annot, 'float32') / K.cast(num_annot, 'float32')

# -----------------------------------------

METRICS = [acc_labeled_only]


## Build model (compile)

In [ ]:
print("compiling model...")

OPTIMIZER = tf.keras.optimizers.Adam(lr) # (Stoian, Schmitt, Scepanovic, Hua)
# OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

# ********************* function to initialize model ********************

def get_model():
	# get_unet_mlp_ts(n_ch, n_timesteps, n_output, patch_height, patch_width, hidden_layers)
	m = get_unet_mlp_ts(len(BANDS),1,CLASS_NUMBER,KERNEL_SIZE,KERNEL_SIZE,[200,100,50])
	m.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
	return m

m_stoian = get_model()
m_stoian.summary()

model = m_stoian

print("...done!")


## Load a model from latest checkpoint

In [ ]:
print("loading weights from trained model...")

# m_updated = get_model()

# get the latest (best) model saved

if (STORAGE == 'DRIVE'):
  path = BASE_FOLDER + FOLDER
else:
  path =  'gs://' + BUCKET + '/' + FOLDER

latest = tf.train.latest_checkpoint(path)
model.load_weights(latest)

print("...done!")

## Set variables

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/n-verde' # INSERT YOUR GEE FOLDER HERE.

# Base file name to use for TFRecord files and assets.
image_base = '1542_CLC_pred_'

# Half this will extend on the sides of each patch.
kernel_buffer = [0, 0]


In [ ]:
# mask image to Kapos mountain area
# image4pred = image_res.mask(kapos)
image4pred = image_res

## Export image for prediction
as TFRecord in Google Cloud. Takes approx **5-13m** per tile (all Greece it takes around **30m**).

In [ ]:
def doExport(out_image_base, kernel_buffer, region, tileNum):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image4pred.select(BANDS),
    description = out_image_base + str(tileNum) + '_',
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base + str(tileNum) + '_',
    region = region.getInfo()['coordinates'],
    scale = pixelSize,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

#   # Block until the task completes.
#   print('Running image export to Google Drive...')
#   import time
#   while task.active():
#     time.sleep(30)

#   # Error condition
#   if task.status()['state'] != 'COMPLETED':
#     print('Error with image export.')
#   else:
#     print('Image export completed.')

In [ ]:
# Run the export for tiles of Greece

# A) loop
tile_number = int(0) # 1st tile
for i in range(0,(len(lis.getInfo()))):

# # B) if you want to run for specific tiles
# tilesList = [149,150,151,745,746]
# for i in tilesList: #

  tile_number = int(i)

  # # option a. without buffer
  # tile = ee.Feature(ee.List(lis.get(i))).geometry().getInfo()
  # region = tile["coordinates"]
  # # print(region)

  # option b. with buffer
  # buffer the tile 2560m to avoid data gaps between tiles
  # the distance of 2560m was found empirically by running the code and seeing the gaps
  prj = ee.Projection('EPSG:3035');  # European projection
  buffer = ee.Feature(ee.List(lis.get(i))).geometry().buffer(2560,None,prj).bounds(0.1)
  buffered_tile = buffer.getInfo()
  buffered_region = buffered_tile["coordinates"]
  region = buffered_region
  # print(region)

  tile_region = ee.Geometry.Polygon(region, None, False)

  # convert "1" to "01"
  tile_number_string = str(tile_number).zfill(3)

  doExport(image_base, kernel_buffer, tile_region, tile_number_string)

  print('done exporting for tile: ', tile_number_string, region)
  tile_number = tile_number

  print('OK!')

## Run prediction
Run the prediction and upload to GEE (for each tile (around 50 tiles), *prediction* takes **~3m** and *upload* to GEE takes around **2h**).


In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  # __________________________________

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}
  # Get only the files generated by the image export for the specific tile.
  exportFilesList = [s for s in filesList if out_image_base in s]
  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f
  # Make sure the files are in the right order.
  imageFilesList.sort()
  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)
  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # __________________________________

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  patch_width = KERNEL_SHAPE[0] + kernel_buffer[0]
  patch_height = KERNEL_SHAPE[1] + kernel_buffer[1]

  buffered_shape = [
      patch_width,
      patch_height]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # __________________________________

  # Perform inference.
  print('Running predictions...')
  predictions = model.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions)

  # # WRITE PREDICTIONS TO STORAGE
  # storage_client = storage.Client()
  # bucket = storage_client.bucket(BUCKET) # bucket_name = "your-bucket-name"
  # blob = bucket.blob(FOLDER + '/' + 'predictions.p') # blob_name = "storage-object-name"
  # with blob.open("wb") as f: # Mode can be specified as wb/rb for bytes mode.
  #     pickle.dump(predictions, f)

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)

  # __________________________________

  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    # print(predictionPatch.shape) # (256, 256, 6)

    # predictionPatch = np.reshape(predictionPatch,(KERNEL_SIZE,KERNEL_SIZE,predictionPatch.shape[-1]))
    # print(predictionPatch.shape) # (256, 256, 6)

    # Create an example.
    example = tf.train.Example(
        features=tf.train.Features(
          feature={
            'pred': tf.train.Feature(
                float_list=tf.train.FloatList(
                    value=np.argmax(predictionPatch, axis=2).flatten())),
          }
        )
      )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

  print('DONE!')

In [ ]:
# Run the prediction for tiles of Greece

# just to invoke earth engine so it doesn't disconnect
try:
  import ee
  ee.Initialize()
except:
  import ee
  ee.Authenticate()
  ee.Initialize()


# A) loop
tile_number = int(0) # 1st tile
for i in range(0,(len(lis.getInfo()))):

# # B) if you want to run for specific tiles
# tilesList = [149,150,151,745,746]
# for i in tilesList: #

  tile_number = int(i)

  # # option a. without buffer
  # tile = ee.Feature(ee.List(lis.get(i))).geometry().getInfo()
  # region = tile["coordinates"]
  # # print(region)

  # option b. with buffer
  # buffer the tile 2560m to avoid data gaps between tiles
  # the distance of 2560m was found empirically by running the code and seeing the gaps
  prj = ee.Projection('EPSG:3035');  # European projection
  buffer = ee.Feature(ee.List(lis.get(i))).geometry().buffer(2560,None,prj).bounds(0.1)
  buffered_tile = buffer.getInfo()
  buffered_region = buffered_tile["coordinates"]
  region = buffered_region
  # print(region)

  tile_region = ee.Geometry.Polygon(region, None, False)

  # convert "1" to "01"
  tile_number_string = str(tile_number).zfill(3)

  image_base2 = image_base + str(tile_number_string)

  doPrediction(image_base2, user_folder, kernel_buffer, tile_region)

  print('done exporting for tile: ', i, region)
  tile_number = tile_number
